In [71]:
#文書を渡すとjanomeで処理してリストで返す関数
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *
from janome.tokenfilter import *
import pathlib
import pandas as pd 

#名詞内の数字をゼロに置き換えるフィルター
class NumReplaceFilter(TokenFilter):

    def apply(self, tokens):
        for token in tokens:
            parts = token.part_of_speech.split(',')
            if (parts[0] == '名詞' and parts[1] == '数'):
                token.surface = '0'
                token.base_form = '0'
                token.reading = 'ゼロ'
                token.phonetic = 'ゼロ'
            yield token

#一文字の単語を除去するフィルター
class OneCharReplaceFilter(TokenFilter):

    def apply(self, tokens):
        for token in tokens:
            # 一文字の単語を除去
            if re.match('^[あ-んア-ンa-zA-Z0-9ー]$', token.surface):
                continue

            yield token

class Kaiseki:

    #ユーザ辞書からneologdを取り込む
    tokenizer = Tokenizer('neologd')

    #Unicode正規化・半角記号除去・全角記号除去・＜＞タグ除去・改行除去
    char_filters = [UnicodeNormalizeCharFilter('NFKC'),
                    RegexReplaceCharFilter('[!/:;%#\@,_✨✅♪\'\$&\?\(\)~\.=\+\-…]+', ''),
                    RegexReplaceCharFilter('[『！”＃＄％＆’（）＝～｜‘｛＋＊｝＜＞？＿－＾￥＠「；：」、。・]', ''),
                    RegexReplaceCharFilter('<.*?>', ''), 
                    RegexReplaceCharFilter('[\n|\r|\t]', '')
                   ]
    #名詞だけ取得・アルファベット小文字化・基本形
    token_filters = [NumReplaceFilter(),
                     OneCharReplaceFilter(),
                     POSKeepFilter(['名詞']),
                     LowerCaseFilter(),
                     ExtractAttributeFilter('base_form')]
    #設定からフィルターを作る
    analyzer = Analyzer(char_filters=char_filters, token_filters=token_filters, tokenizer=tokenizer)    

    #処理中止ワードを読み込む
    ban_words = []
    path = 'ban_word.txt'
    with open(path, encoding="utf-8_sig") as f:
        ban_words = f.read().split('\n')    
    
    #ストップワードを読み込む
    stop_words = []
    path = 'Japanese.txt'
    with open(path, encoding="utf-8_sig") as f:
        stop_words = f.read().split('\n')


    def __init__(self):
        pass
        
    def before_filter(self, before_st):
        
        after_st = []
        
        #禁止ワードが含まれている場合、処理を行わず終了する
        for ban_word in self.ban_words:
            #文字列に禁止ワードが含まれているか判定
            if ban_word in before_st:
                #含まれていた場合、空で返す
                return after_st      

        #文章をanalyzerに掛け処理する
        after_st = self.analyzer.analyze(before_st)

        after_st = [x for x in after_st if x not in self.stop_words]

        return after_st



In [27]:
import pathlib
import pandas as pd 
import numpy as np

def wordCount(folder, file):
    #CSVを読み込む
    path = "" #ファイルを直下に置いてない場合はパスを入れる
    filetest = folder + file
    
    #pandasでCSVファイルを開き、3列目のツイート内容だけdfに取得する
    df = pd.read_csv(filetest, usecols=[2], encoding='utf-8', engine='python')
    
    
    #単語のカウント表を作る
    dfCount = pd.DataFrame(columns=['count'])
    
    #解析クラスを用意
    kaiseki = Kaiseki()
    
    #ツイート本文を形態素解析し、名詞・動詞・形容詞だけ抽出したリストを作成する
    num = 0
    for row in df['ツイート']:
        word_list = kaiseki.before_filter(row)
        for word in word_list:
            #リストを頻出単度をカウントする辞書に追加する、辞書になければ追加する
            if not any(dfCount.index == word): 
                dfCount.loc[word] = 1
            else:
                dfCount.at[word, 'count'] += 1
    
    #得点表を降順ソート
    count_sorted = dfCount.sort_values('count', ascending=False)
    
    #結果を保存
    count_sorted.to_csv(folder+ 'Count_' + file)
    

In [20]:
wordCount('paypay/', 'paypay_0615_0621_with_sentiment.csv')

In [30]:
#ネガポジが付与されたデータの単語を集計する

#引数：フォルダ、ファイル名

import pathlib
import pandas as pd 
import numpy as np

def wordCount_sent(folder, file):
    #CSVを読み込む
    path = "" #ファイルを直下に置いてない場合はパスを入れる
    filename = folder + file
    
    #pandasでCSVファイルを開く
    df = pd.read_csv(filename, usecols=[2,13], encoding='utf8', engine='python')

    #SENTIMENT列が0のネガティブツイートの3列目のツイート内容をdfに取得する
    negative_df = df[df['SENTIMENT'] == 0]
    
    #SENTIMENT列が1のポジティブツイートの3列目のツイート内容をdfに取得する
    positive_df = df[df['SENTIMENT'] == 1]
    
    #ネガティブツイートの内容を解析し、単語のカウントリストを作成する
    result_countlist_negative = create_countlist(negative_df)
    #結果を保存
    result_countlist_negative.to_csv(folder+ 'Count_negative_' + file)
    
    #ポジティブツイートの内容を解析し、単語のカウントリストを作成する
    result_countlist_positive = create_countlist(positive_df)
    
    #結果を保存
    result_countlist_positive.to_csv(folder+ 'Count_positive_' + file)
    


#Dataframeを受け取り、ツイート列の内容を形態素解析、結果をCSVで出力する処理
def create_countlist(df):
    
    #解析クラスを用意
    kaiseki = Kaiseki()
    
    #単語のカウント表を作る
    dfCount = pd.DataFrame(columns=['count'])
    
    #ツイート本文を形態素解析し、名詞・動詞・形容詞だけ抽出したリストを作成する
    num = 0
    for row in df['ツイート']:
        word_list = kaiseki.before_filter(row)
        for word in word_list:
            #リストを頻出単度をカウントする辞書に追加する、辞書になければ追加する
            if not any(dfCount.index == word): 
                dfCount.loc[word] = 1
            else:
                dfCount.at[word, 'count'] += 1
    
    #得点表を降順ソート
    count_sorted = dfCount.sort_values('count', ascending=False)
    
    return count_sorted
    
    

In [32]:
wordCount_sent('sentiment/', 'paypay_0615_0621_with_sentiment.csv')

KeyboardInterrupt: 

In [73]:
wordCount_sent('sentiment/data/', 'paypay_0608_0614_with_sentiment.csv')
wordCount_sent('sentiment/data/', 'paypay_0615_0621_with_sentiment.csv')
wordCount_sent('sentiment/data/', 'paypay_0622_0628_with_sentiment.csv')
wordCount_sent('sentiment/data/', 'paypay_0629_0705_with_sentiment.csv')

In [72]:
wordCount_sent('sentiment/data/', 'paypay_0601_0607_with_sentiment.csv')